In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
%cd "/content/drive/MyDrive/112_2_DL"
import os
os.listdir()

/content/drive/MyDrive/112_2_DL


['Assignment2_Q1.ipynb', 'images', 'Assignment2_Q2.ipynb']

In [1]:
import pandas as pd
import numpy as np
import cv2

In [2]:
train_set = pd.read_csv("images/train.txt", sep=" ", header=None)
valid_set = pd.read_csv("images/val.txt", sep=" ", header=None)
test_set = pd.read_csv("images/test.txt", sep=" ", header = None)

In [3]:
train_set = train_set.rename(columns = {0:"image", 1:"class"})
train_set

,image,class
0,images/n02111277/n02111277_10005.JPEG,0
1,images/n02111277/n02111277_10019.JPEG,0
2,images/n02111277/n02111277_10072.JPEG,0
3,images/n02111277/n02111277_1008.JPEG,0
4,images/n02111277/n02111277_10089.JPEG,0
...,...,...
63320,images/n02172182/n02172182_9515.JPEG,49
63321,images/n02172182/n02172182_954.JPEG,49
63322,images/n02172182/n02172182_9549.JPEG,49
63323,images/n02172182/n02172182_9589.JPEG,49


In [4]:
valid_set = valid_set.rename(columns = {0:"image", 1:"class"})
valid_set

,image,class
0,images/n02111277/n02111277_9695.JPEG,0
1,images/n02111277/n02111277_9747.JPEG,0
2,images/n02111277/n02111277_975.JPEG,0
3,images/n02111277/n02111277_9795.JPEG,0
4,images/n02111277/n02111277_980.JPEG,0
...,...,...
445,images/n02172182/n02172182_993.JPEG,49
446,images/n02172182/n02172182_9953.JPEG,49
447,images/n02172182/n02172182_997.JPEG,49
448,images/n02172182/n02172182_998.JPEG,49


In [5]:
test_set = test_set.rename(columns = {0:"image", 1:"class"})
test_set

,image,class
0,images/n02111277/n02111277_9420.JPEG,0
1,images/n02111277/n02111277_9422.JPEG,0
2,images/n02111277/n02111277_9484.JPEG,0
3,images/n02111277/n02111277_951.JPEG,0
4,images/n02111277/n02111277_9518.JPEG,0
...,...,...
445,images/n02172182/n02172182_974.JPEG,49
446,images/n02172182/n02172182_9765.JPEG,49
447,images/n02172182/n02172182_9789.JPEG,49
448,images/n02172182/n02172182_98.JPEG,49


In [6]:
train_img = []
for i in range(len(train_set["image"])):
    temp = cv2.imread("images/"+train_set["image"][i])
    temp = cv2.resize(temp, (128, 128))
    train_img.append(temp)

In [7]:
valid_img = []
for i in range(len(valid_set["image"])):
    temp = cv2.imread("images/"+valid_set["image"][i])
    temp = cv2.resize(temp, (128, 128))
    valid_img.append(temp)

In [8]:
test_img = []
for i in range(len(test_set["image"])):
    temp = cv2.imread("images/"+test_set["image"][i])
    temp = cv2.resize(temp, (128, 128))
    test_img.append(temp)

In [9]:
train_img = np.array(train_img)
valid_img = np.array(valid_img)
test_img = np.array(test_img)

In [10]:
import torch

In [11]:
x_train = torch.from_numpy(train_img)
x_valid = torch.from_numpy(valid_img)
x_test = torch.from_numpy(test_img)

In [12]:
x_train = x_train.permute(0,3,1,2)
x_valid = x_valid.permute(0,3,1,2)
x_test = x_test.permute(0,3,1,2)

In [13]:
y_train = train_set["class"]
y_valid = valid_set["class"]
y_test = test_set["class"]

In [14]:
import torch.nn as nn
import torch.nn.functional as F

In [15]:
class attention2d(nn.Module):
    def __init__(self, in_planes, ratios, K, temperature, init_weight=True):
        super(attention2d, self).__init__()
        assert temperature%3==1
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        if in_planes!=3:
            hidden_planes = int(in_planes*ratios)+1
        else:
            hidden_planes = K
        self.fc1 = nn.Conv2d(in_planes, hidden_planes, 1, bias=False)
        # self.bn = nn.BatchNorm2d(hidden_planes)
        self.fc2 = nn.Conv2d(hidden_planes, K, 1, bias=True)
        self.temperature = temperature
        if init_weight:
            self._initialize_weights()


    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            if isinstance(m ,nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def updata_temperature(self):
        if self.temperature!=1:
            self.temperature -=3
            #print('Change temperature to:', str(self.temperature))


    def forward(self, x):
        x = self.avgpool(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x).view(x.size(0), -1)
        return F.softmax(x/self.temperature, 1)

In [16]:
class Dynamic_conv2d(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, ratio=0.25, stride=1, padding=0, dilation=1, groups=1, bias=True, K=4,temperature=34, init_weight=True):
        super(Dynamic_conv2d, self).__init__()
        assert in_planes%groups==0
        self.in_planes = in_planes
        self.out_planes = out_planes
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.groups = groups
        self.bias = bias
        self.K = K
        self.attention = attention2d(in_planes, ratio, K, temperature)

        self.weight = nn.Parameter(torch.randn(K, out_planes, in_planes//groups, kernel_size, kernel_size), requires_grad=True)
        if bias:
            self.bias = nn.Parameter(torch.zeros(K, out_planes))
        else:
            self.bias = None
        if init_weight:
            self._initialize_weights()

        #TODO 初始化
    def _initialize_weights(self):
        for i in range(self.K):
            nn.init.kaiming_uniform_(self.weight[i])


    def update_temperature(self):
        self.attention.updata_temperature()

    def forward(self, x):#将batch视作维度变量，进行组卷积，因为组卷积的权重是不同的，动态卷积的权重也是不同的
        softmax_attention = self.attention(x)
        batch_size, in_planes, height, width = x.size()
        x = x.view(1, -1, height, width)# 变化成一个维度进行组卷积
        weight = self.weight.view(self.K, -1)

        # 动态卷积的权重的生成， 生成的是batch_size个卷积参数（每个参数不同）
        aggregate_weight = torch.mm(softmax_attention, weight).view(batch_size*self.out_planes, self.in_planes//self.groups, self.kernel_size, self.kernel_size)
        if self.bias is not None:
            aggregate_bias = torch.mm(softmax_attention, self.bias).view(-1)
            output = F.conv2d(x, weight=aggregate_weight, bias=aggregate_bias, stride=self.stride, padding=self.padding,
                              dilation=self.dilation, groups=self.groups*batch_size)
        else:
            output = F.conv2d(x, weight=aggregate_weight, bias=None, stride=self.stride, padding=self.padding,
                              dilation=self.dilation, groups=self.groups * batch_size)

        output = output.view(batch_size, self.out_planes, output.size(-2), output.size(-1))
        return output

In [17]:
def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    return Dynamic_conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=dilation, groups=groups, bias=False, dilation=dilation)

In [18]:
def conv1x1(in_planes, out_planes, stride=1):
    return Dynamic_conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False,)

In [19]:
class BasicBlock(nn.Module):
    expansion = 1
    __constants__ = ['downsample']

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [20]:
class Bottleneck(nn.Module):
    expansion = 4
    __constants__ = ['downsample']

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [21]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=50, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def update_temperature(self):
        for m in self.modules():
            if isinstance(m, Dynamic_conv2d):
                m.update_temperature()


    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)

In [22]:
def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model

In [23]:
def dy_resnet18(pretrained=False, progress=True, **kwargs):
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', BasicBlock, [2, 2, 2, 2], pretrained, progress,
                   **kwargs)

In [24]:
from torchvision.models.resnet import resnet18 as raw_resnet18

In [25]:
numclasses = 50
model = dy_resnet18(num_classes=numclasses)
net_name = "dy_resnet18"

import argparse


parser = argparse.ArgumentParser(description='dynamic convolution')

parser.add_argument('--dataset', type=str, default='mini-ImageNet', help='training dataset')

parser.add_argument('--batch-size', type=int, default=128)

parser.add_argument('--test-batch-size', type=int, default=20)

parser.add_argument('--epochs', type=int, default=160)

parser.add_argument('--lr', type=float, default=0.1, )

parser.add_argument('--momentum', type=float, default=0.9)

parser.add_argument('--weight-decay', type=float, default=1e-4)

parser.add_argument('--net-name', default='dy_resnet18')


args = parser.parse_args()

args.device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [26]:
lr = 0.001
momentum = 0.9
weight_decay = 1e-4
epochs = 30
batch_size = 128
valid_batch_size = 20
test_batch_size = 20

In [27]:
import torch.optim as optim

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
# loader = DataLoader(list(zip(X_train, y_train)), shuffle=True, batch_size=16)
trainloader = torch.utils.data.DataLoader(list(zip(x_train, y_train)), batch_size=batch_size, shuffle=True)
validloader = torch.utils.data.DataLoader(list(zip(x_valid, y_valid)), batch_size=valid_batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(list(zip(x_test, y_test)), batch_size=test_batch_size, shuffle=False)
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

In [28]:
def adjust_lr(optimizer, epoch):
    if epoch in [epochs*0.5, epochs*0.75, epochs*0.85]:
        for p in optimizer.param_groups:
            p['lr'] *= 0.1
            lr = p['lr']
        #print('Change lr:'+str(lr))

In [29]:
def train(epoch):
    model.train()
    avg_loss = 0.
    train_acc = 0.
    adjust_lr(optimizer, epoch)
    for batch_idx, (data, target) in enumerate(trainloader):

        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data.to(torch.float32))
        loss = F.cross_entropy(output, target)
        avg_loss += loss.item()
        pred = output.data.max(1, keepdim=True)[1]
        train_acc += pred.eq(target.data.view_as(pred)).cpu().sum()
        loss.backward()

        optimizer.step()
    print('Train Epoch: {}, loss{:.6f}, acc{}'.format(epoch, loss.item(), train_acc/len(trainloader.dataset)), end='')
    if net_name.startswith('dy'):
        model.update_temperature()

In [30]:
def validation(epoch):
    model.eval()
    valid_loss = 0.
    correct=0.
    with torch.no_grad():
        for data, label in validloader:
            data, label = data.to(device), label.to(device)
            output = model(data.to(torch.float32))
            valid_loss += F.cross_entropy(output, label, size_average=False).item()
            pred =  output.data.max(1, keepdim=True)[1]
            correct += pred.eq(label.data.view_as(pred)).cpu().sum()
    valid_loss/=len(validloader.dataset)
    correct = int(correct)
    print('Valid set:average loss: {:.4f}, accuracy{}'.format(valid_loss, correct/len(validloader.dataset)))
    return correct/len(validloader.dataset)

In [31]:
best_val_acc=0.
val_acc = []
for i in range(epochs):
    train(i+1)
    temp_acc = validation(i+1)
    val_acc.append(temp_acc)
    if temp_acc > best_val_acc:
        best_val_acc = temp_acc
print('Best acc{}'.format(best_val_acc))

Train Epoch: 1, loss3.496431, acc0.07297275960445404

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Valid set:average loss: 3.5395, accuracy0.10222222222222223
Train Epoch: 2, loss3.042214, acc0.14525069296360016Valid set:average loss: 3.4573, accuracy0.12444444444444444
Train Epoch: 3, loss3.042984, acc0.18869325518608093Valid set:average loss: 3.4093, accuracy0.14
Train Epoch: 4, loss2.613018, acc0.22160284221172333Valid set:average loss: 3.1174, accuracy0.1711111111111111
Train Epoch: 5, loss2.686348, acc0.2497749775648117Valid set:average loss: 3.0819, accuracy0.17777777777777778
Train Epoch: 6, loss2.675732, acc0.2753888666629791Valid set:average loss: 2.6110, accuracy0.25555555555555554
Train Epoch: 7, loss2.685779, acc0.3032451570034027Valid set:average loss: 2.8486, accuracy0.23777777777777778
Train Epoch: 8, loss2.596398, acc0.32669562101364136Valid set:average loss: 3.1939, accuracy0.21555555555555556
Train Epoch: 9, loss2.440928, acc0.3447927236557007Valid set:average loss: 4.9922, accuracy0.17555555555555555
Train Epoch: 10, loss2.362315, acc0.2915436327457428Valid set:av

In [31]:
def test():
    model.eval()
    test_loss = 0.
    correct=0.
    with torch.no_grad():
        for data, label in testloader:
            data, label = data.to(device), label.to(device)
            output = model(data.to(torch.float32))
            test_loss += F.cross_entropy(output, label, size_average=False).item()
            pred =  output.data.max(1, keepdim=True)[1]
            correct += pred.eq(label.data.view_as(pred)).cpu().sum()
    test_loss/=len(testloader.dataset)
    correct = int(correct)
    print('Test set:average loss: {:.4f}, accuracy{}'.format(test_loss, 100.*correct/len(testloader.dataset)))
    return correct/len(testloader.dataset)

In [33]:
test()

Test set:average loss: 3.2924, accuracy17.11111111111111


0.1711111111111111

In [32]:
from torchvision import models

model_resnet18 = models.resnet18(pretrained = False)
num_feature = model_resnet18.fc.in_features
model_resnet18.fc = torch.nn.Linear(num_feature, numclasses)
model_resnet18.to(device)
optimizer_resnet = optim.SGD(model_resnet18.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [33]:
def train_resnet(epoch):
    model_resnet18.train()
    avg_loss = 0.
    train_acc = 0.
    adjust_lr(optimizer_resnet, epoch)
    for batch_idx, (data, target) in enumerate(trainloader):

        data, target = data.to(device), target.to(device)
        optimizer_resnet.zero_grad()
        output = model_resnet18(data.to(torch.float32))
        loss = F.cross_entropy(output, target)
        avg_loss += loss.item()
        pred = output.data.max(1, keepdim=True)[1]
        train_acc += pred.eq(target.data.view_as(pred)).cpu().sum()
        loss.backward()

        optimizer_resnet.step()
    print('Train Epoch: {}, loss{:.6f}, acc{}'.format(epoch, loss.item(), train_acc/len(trainloader.dataset)), end='')

In [34]:
def validation_resnet(epoch):
    model_resnet18.eval()
    valid_loss = 0.
    correct=0.
    with torch.no_grad():
        for data, label in validloader:
            data, label = data.to(device), label.to(device)
            output = model_resnet18(data.to(torch.float32))
            valid_loss += F.cross_entropy(output, label, size_average=False).item()
            pred =  output.data.max(1, keepdim=True)[1]
            correct += pred.eq(label.data.view_as(pred)).cpu().sum()
    valid_loss/=len(validloader.dataset)
    correct = int(correct)
    print('Valid set:average loss: {:.4f}, accuracy{}'.format(valid_loss, correct/len(validloader.dataset)))
    return correct/len(validloader.dataset)

In [35]:
best_val_acc_18=0.
val_acc_18 = []
for i in range(epochs):
    train_resnet(i+1)
    temp_acc = validation_resnet(i+1)
    val_acc_18.append(temp_acc)
    if temp_acc > best_val_acc_18:
        best_val_acc_18 = temp_acc
print('Best acc{}'.format(best_val_acc_18))

Train Epoch: 1, loss3.331717, acc0.07848401367664337

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Valid set:average loss: 3.6017, accuracy0.10444444444444445
Train Epoch: 2, loss3.092461, acc0.15172523260116577Valid set:average loss: 3.1083, accuracy0.2
Train Epoch: 3, loss2.831559, acc0.19603632390499115Valid set:average loss: 3.0062, accuracy0.16444444444444445
Train Epoch: 4, loss2.745202, acc0.23069877922534943Valid set:average loss: 2.8775, accuracy0.2111111111111111
Train Epoch: 5, loss2.843068, acc0.26406633853912354Valid set:average loss: 2.7759, accuracy0.2111111111111111
Train Epoch: 6, loss2.265332, acc0.2912119925022125Valid set:average loss: 3.3361, accuracy0.14666666666666667
Train Epoch: 7, loss2.376279, acc0.3178681433200836Valid set:average loss: 2.6751, accuracy0.25333333333333335
Train Epoch: 8, loss1.974718, acc0.3444611132144928Valid set:average loss: 2.6476, accuracy0.25555555555555554
Train Epoch: 9, loss2.341848, acc0.36901697516441345Valid set:average loss: 3.2822, accuracy0.21333333333333335
Train Epoch: 10, loss1.985300, acc0.39379391074180603Valid set:av

: 

In [ ]:
def test_resnet():
    model_resnet18.eval()
    test_loss = 0.
    correct=0.
    with torch.no_grad():
        for data, label in testloader:
            data, label = data.to(device), label.to(device)
            output = model_resnet18(data.to(torch.float32))
            test_loss += F.cross_entropy(output, label, size_average=False).item()
            pred =  output.data.max(1, keepdim=True)[1]
            correct += pred.eq(label.data.view_as(pred)).cpu().sum()
    test_loss/=len(testloader.dataset)
    correct = int(correct)
    print('Test set:average loss: {:.4f}, accuracy{}'.format(test_loss, 100.*correct/len(testloader.dataset)))
    return correct/len(testloader.dataset)

In [ ]:
test_resnet()